### Goals
- Build a movie recommender system

In [23]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# Loading dataset
movies_df = pd.read_csv("./ml-latest/movies.csv")
# links_df = pd.read_csv("./ml-latest/links.csv")
ratings_df = pd.read_csv("./ml-latest/ratings.csv")
# tags_df= pd.read_csv("./ml-latest/tags.csv")
movies_df.head()
print(ratings_df.head())

   userId  movieId  rating   timestamp
0       1      169     2.5  1204927694
1       1     2471     3.0  1204927438
2       1    48516     5.0  1204927435
3       2     2571     3.5  1436165433
4       2   109487     4.0  1436165496


In [25]:
movies_df[movies_df["title"] == "Gladiator (2000)"]

,movieId,title,genres
3488,3578,Gladiator (2000),Action|Adventure|Drama


In [26]:
print(movies_df.shape)
print(movies_df.size)
# print(movies_df.dtypes)
# print(movies_df.info())
# print(movies_df.describe())
print(ratings_df.shape)
print(ratings_df.size)
# print(ratings_df.dtypes)

(34208, 3)
102624
(22884377, 4)
91537508


In [27]:
temp_df = movies_df.head().copy()
temp_df["title"]
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [28]:
# Preprocessing
# Collaborative filtering
# Constest based filtering
# Movie dataset from Kaggle
# Preprocessing
# cleaning the data
# Understand how to do collaborative filtering
# By using similarity score 
# print(movies_df["title"])
# Cleaning movies data
movies_df["year"] = movies_df.title.str.extract("(\(\d{4}\))", expand=False)
movies_df["year"] = movies_df.year.str.extract("(\d{4})", expand=False)
movies_df["title"] = movies_df.title.str.replace(r"(\(\d{4}\))", "", regex=True)
movies_df["title"] = movies_df["title"].apply(lambda x: x.strip())
movies_df = movies_df.drop("genres", axis=1)
movies_df

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
34203,151697,Grand Slam,1967
34204,151701,Bloodmoney,2010
34205,151703,The Butterfly Circus,2009
34206,151709,Zero,2015


In [29]:
# ratings data cleaning
ratings_df= ratings_df.drop("timestamp", axis=1)
ratings_df.head()


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


- Collaborating filtering to predict ratings
- Using Pearson Correlation Function

$$

$$

In [30]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [42]:
# find the row present in the movies based on the title present in the userInput
# then add that data to input
# removed not required field
inputMovies = pd.DataFrame(userInput)
inputId = movies_df[movies_df["title"].isin(inputMovies["title"].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop(["year"], axis=1)
inputMovies.shape
print(inputMovies)
# movies_df.head()
# ratings_df.head()

   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0


In [43]:
# Users that watched same movies as inputMovies
users = ratings_df[ratings_df["movieId"].isin(inputMovies["movieId"].tolist())]
users.shape
users.head()


,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [44]:
# group the rows by userId
userSubset = users.groupby(["userId"])
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
22883679,247738,296,4.0
22884132,247751,1,4.0
22884142,247751,296,4.0
22884164,247751,1274,5.0


In [45]:
# Looking into the users
userSubset.get_group(1130)

/tmp/ipykernel_22101/1936040501.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  userSubset.get_group(1130)


,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


In [46]:
# sorting the users with most in common
userSubset = sorted(userSubset, key=lambda x: len(x[1]), reverse=True)
userSubset[0:3]

[((75,),
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 ((106,),
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 ((686,),
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

In [64]:
# Similarity of users to input users
userSubsetGroup = userSubset[0:100]
userSubsetGroup

[((75,),
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 ((106,),
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 ((686,),
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0),
 ((815,),
         userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73922     815      296     5.0
  74362     815     1274     3.0
  74678     815     1968     4.5),
 ((1040,),
         userId  movieId  rating
  96689    1040        1     3.0
  96690    1040        2     1.5
  96733    1040      296     3.5
  96859    1040     1274     3

In [51]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    print(temp_df.head())
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0
   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0
   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0
   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0
   movieId                title  rating


In [53]:
pearsonCorrelationDict.items()

dict_items([((75,), 0.8272781516947562), ((106,), 0.5860090386731182), ((686,), 0.8320502943378437), ((815,), 0.5765566601970551), ((1040,), 0.9434563530497265), ((1130,), 0.2891574659831201), ((1502,), 0.8770580193070299), ((1599,), 0.4385290096535153), ((1625,), 0.716114874039432), ((1950,), 0.179028718509858), ((2065,), 0.4385290096535153), ((2128,), 0.5860090386731196), ((2432,), 0.1386750490563073), ((2791,), 0.8770580193070299), ((2839,), 0.8204126541423674), ((2948,), -0.11720180773462392), ((3025,), 0.45124262819713973), ((3040,), 0.89514359254929), ((3186,), 0.6784622064861935), ((3271,), 0.26989594817970664), ((3429,), 0.0), ((3734,), -0.15041420939904673), ((4099,), 0.05860090386731196), ((4208,), 0.29417420270727607), ((4282,), -0.4385290096535115), ((4292,), 0.6564386345361464), ((4415,), -0.11183835382312353), ((4586,), -0.9024852563942795), ((4725,), -0.08006407690254357), ((4818,), 0.4885967564883424), ((5104,), 0.7674257668936507), ((5165,), -0.4385290096535153), ((554

In [78]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
print(pearsonDF.index[0][0])
# for i in range(len(pearsonDF.index)):
#     pearsonDF.index[i] = pearsonDF.index[i][0]
pearsonDF['userId'] = pearsonDF.index#[0][0]
pearsonDF.index = range(len(pearsonDF))
pearsonDF['userId'] = pearsonDF["userId"].apply(lambda x : x[0]).astype("int64")
pearsonDF.head()

75


,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


In [79]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040


In [80]:
print(ratings_df.head())
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

   userId  movieId  rating
0       1      169     2.5
1       1     2471     3.0
2       1    48516     5.0
3       2     2571     3.5
4       2   109487     4.0


,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5


In [82]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [83]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [85]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


In [87]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5073,5.0,5073
2284,5.0,2284
3329,5.0,3329
28,5.0,28
945,5.0,945
2848,5.0,2848
987,5.0,987
1024,5.0,1024
55067,5.0,55067


In [89]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
27,28,Persuasion,1995
928,945,Top Hat,1935
969,987,Bliss,1997
1005,1024,"Three Caballeros, The",1945
2200,2284,Bandit Queen,1994
2763,2848,Othello (Tragedy of Othello: The Moor of Venic...,1952
3243,3329,"Year My Voice Broke, The",1987
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
9987,32792,"Red Desert (Deserto rosso, Il)",1964
12111,55067,Requiem,2006


In [90]:
# This project is referenced from IBM ML course

In [ ]:
# Summary
# Collaborative filtering